In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
print('This notebook was run on ' + time.strftime("%d/%m/%Y") + ' at ' + time.strftime("%H:%M:%S") + '.')

This notebook was run on 06/11/2023 at 21:46:22.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
dataset_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'dataset')
print('Dataset directory: {}'.format(dataset_directory))

dataset_file = 'dataset.csv'
print('Dataset file: {}'.format(dataset_file))

dataset_path = os.path.join(dataset_directory, dataset_file)
print('Dataset path: {}'.format(dataset_path))

# ======================================

model_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'model')
print('Model directory: {}'.format(model_directory))

model_file = 'cm2pixel_model.pt'
print('Model file: {}'.format(model_file))

model_path = os.path.join(model_directory, model_file)
print('Model path: {}'.format(model_path))

Dataset directory: /home/icar/icar-ng-data/dataset
Dataset file: dataset.csv
Dataset path: /home/icar/icar-ng-data/dataset/dataset.csv
Model directory: /home/icar/icar-ng-data/model
Model file: cm2pixel_model.pt
Model path: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [5]:
# Check whether dataset file exists
if not os.path.exists(dataset_path):
    print('Dataset file does not exist: {}'.format(dataset_path))
    exit()

# Check whether model directory exists
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
    print('Created model directory: {}'.format(model_directory))

In [6]:
df_raw = pd.read_csv(dataset_path)
df_train = df_raw.sample(frac=0.8)
df_test = df_raw.drop(df_train.index)

min_x = np.min(df_train.iloc[:, 2:4].values, axis=0)
min_x = torch.tensor(min_x, dtype=torch.float32)
max_x = np.max(df_train.iloc[:, 2:4].values, axis=0)
max_x = torch.tensor(max_x, dtype=torch.float32)
min_y = np.min(df_train.iloc[:, 0:2].values, axis=0)
min_y = torch.tensor(min_y, dtype=torch.float32)
max_y = np.max(df_train.iloc[:, 0:2].values, axis=0)
max_y = torch.tensor(max_y, dtype=torch.float32)

print('min_x: {}, max_x: {}'.format(min_x, max_x))
print('min_y: {}, max_y: {}'.format(min_y, max_y))

min_x: tensor([   0.0000, -665.6403]), max_x: tensor([1200.0000,  848.5281])
min_y: tensor([ 30., 204.]), max_y: tensor([1272.,  692.])


In [7]:
class ForwardDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        x = self.dataframe.iloc[:, 2:4].values
        y = self.dataframe.iloc[:, 0:2].values
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
dataset_train = ForwardDataset(df_train)
dataset_test = ForwardDataset(df_test)
print('dataset_train: {}'.format(len(dataset_train)))
print('dataset_test: {}'.format(len(dataset_test)))

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True, pin_memory=True)

dataset_train: 118
dataset_test: 29


In [9]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, min_x, max_x, min_y, max_y):
        super(MultiLayerPerceptron, self).__init__()
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 20)
        self.fc3 = nn.Linear(20, 80)
        self.fc4 = nn.Linear(80, 20)
        self.fc5 = nn.Linear(20, 4)
        self.fc6 = nn.Linear(4, output_size)

    def forward(self, x):
        x = (x - self.min_x) / (self.max_x - self.min_x)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.fc6(x)
        x = x * (self.max_y - self.min_y) + self.min_y
        return x

In [10]:
model = MultiLayerPerceptron(2, 2, min_x.to(device), max_x.to(device), min_y.to(device), max_y.to(device)).to(device)

if os.path.exists(model_path):
    try:
        print('Loading model: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
        print('Loaded model: {}'.format(model_path))
    except BaseException as e:
        print('Failed to load model: {}'.format(model_path))
        print(e)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Loading model: /home/icar/icar-ng-data/model/cm2pixel_model.pt
Loaded model: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [11]:
min_test_loss = np.inf

for epoch in range(50000):
    model.train()
    train_loss = 0.0
    for i, (x, y) in enumerate(dataloader_train):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if epoch % 100 == 0:
        model.eval()
        test_loss = 0.0
        for i, (x, y) in enumerate(dataloader_test):
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss += loss.item()

        save_model = False
        if test_loss < min_test_loss:
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_path)
            save_model = True

        if save_model:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f} (Saved)'.format(epoch, train_loss, test_loss))
        else:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f}'.format(epoch, train_loss, test_loss))

epoch: 0, train_loss: 362789.187500, test_loss: 128736.757812 (Saved)
epoch: 100, train_loss: 295456.921875, test_loss: 94952.304688 (Saved)
epoch: 200, train_loss: 249925.171875, test_loss: 78893.031250 (Saved)
epoch: 300, train_loss: 217964.890625, test_loss: 68458.742188 (Saved)
epoch: 400, train_loss: 180029.445312, test_loss: 61140.601562 (Saved)
epoch: 500, train_loss: 157320.835938, test_loss: 55609.027344 (Saved)
epoch: 600, train_loss: 136375.972656, test_loss: 50733.558594 (Saved)
epoch: 700, train_loss: 118432.917969, test_loss: 46996.863281 (Saved)
epoch: 800, train_loss: 103949.160156, test_loss: 43664.476562 (Saved)
epoch: 900, train_loss: 91680.980469, test_loss: 38875.640625 (Saved)
epoch: 1000, train_loss: 77516.207031, test_loss: 33705.539062 (Saved)
epoch: 1100, train_loss: 62097.507812, test_loss: 27999.960938 (Saved)
epoch: 1200, train_loss: 49479.753906, test_loss: 23256.404297 (Saved)
epoch: 1300, train_loss: 42825.185547, test_loss: 20575.015625 (Saved)
epoch: 1

In [12]:
onnx_path = os.path.join(model_directory, 'cm2pixel_model.onnx')
print('ONNX path: {}'.format(onnx_path))

try:
    print('Saving ONNX model: {}'.format(onnx_path))
    torch.onnx.export(model, torch.randn(1, 2).to(device), onnx_path, verbose=True)
    print('Saved ONNX model: {}'.format(onnx_path))
except BaseException as e:
    print('Failed to save ONNX model: {}'.format(onnx_path))
    print(e)

ONNX path: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Saving ONNX model: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Failed to save ONNX model: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Module onnx is not installed!


In [13]:
model.eval()
for i, (x, y) in enumerate(dataloader_test):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    
    print('x\n{}'.format(x))
    print('y\n{}'.format(y))
    print('y_pred\n{}'.format(y_pred))
    print('y - y_pred\n{}'.format(y - y_pred))
    
    break
     

x
tensor([[ 450.0000, -150.0000],
        [ 789.1151,  131.5192],
        [  25.0000,  -50.0000],
        [ 848.5281, -848.5281],
        [ 150.0000,  100.0000],
        [ 447.2136,  223.6068],
        [ 350.0000, -100.0000],
        [ 100.0000, -100.0000],
        [   0.0000,  100.0000],
        [ 416.0251, -277.3501],
        [ 447.2136, -223.6068],
        [ 450.0000, -100.0000],
        [ 350.0000, -250.0000],
        [ 565.6854,  565.6854],
        [ 350.0000, -300.0000],
        [ 200.0000, -100.0000],
        [ 998.4603,  665.6403],
        [ 350.0000, -200.0000],
        [  75.0000,  100.0000],
        [  75.0000,  150.0000],
        [ 350.0000, -150.0000],
        [ 200.0000,  150.0000],
        [   0.0000,  -25.0000],
        [  75.0000,  -50.0000],
        [ 450.0000, -250.0000],
        [ 100.0000, -200.0000],
        [ 758.9467,  252.9822],
        [  75.0000,    0.0000],
        [ 250.0000, -150.0000]], device='cuda:0')
y
tensor([[ 816.,  260.],
        [ 542.,  228.],
  